<a href="https://colab.research.google.com/github/CM-007/DS_ASS/blob/main/Ass15_Random_Forests(Company_Data).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Import Libraries**

In [41]:
import pandas as pd

from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

# Random Forest
from sklearn.ensemble import RandomForestClassifier

# Bagging
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

# Boosting
from sklearn.ensemble import AdaBoostClassifier

# Stacking
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier

import warnings
warnings.filterwarnings("ignore")

# **Read Data Set using pandas**

In [42]:
data = pd.read_csv("Company_Data.csv")

# **EDA**

In [43]:
data.head(10)

,Sales,CompPrice,Income,Advertising,Population,Price,ShelveLoc,Age,Education,Urban,US
0,9.50,138,73,11,276,120,Bad,42,17,Yes,Yes
1,11.22,111,48,16,260,83,Good,65,10,Yes,Yes
2,10.06,113,35,10,269,80,Medium,59,12,Yes,Yes
3,7.40,117,100,4,466,97,Medium,55,14,Yes,Yes
4,4.15,141,64,3,340,128,Bad,38,13,Yes,No
5,10.81,124,113,13,501,72,Bad,78,16,No,Yes
6,6.63,115,105,0,45,108,Medium,71,15,Yes,No
7,11.85,136,81,15,425,120,Good,67,10,Yes,Yes
8,6.54,132,110,0,108,124,Medium,76,10,No,No
9,4.69,132,113,0,131,124,Medium,76,17,No,Yes


In [44]:
data.shape

(400, 11)

In [45]:
data.describe()

,Sales,CompPrice,Income,Advertising,Population,Price,Age,Education
count,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000
mean,7.496325,124.975000,68.657500,6.635000,264.840000,115.795000,53.322500,13.900000
std,2.824115,15.334512,27.986037,6.650364,147.376436,23.676664,16.200297,2.620528
min,0.000000,77.000000,21.000000,0.000000,10.000000,24.000000,25.000000,10.000000
25%,5.390000,115.000000,42.750000,0.000000,139.000000,100.000000,39.750000,12.000000
50%,7.490000,125.000000,69.000000,5.000000,272.000000,117.000000,54.500000,14.000000
75%,9.320000,135.000000,91.000000,12.000000,398.500000,131.000000,66.000000,16.000000
max,16.270000,175.000000,120.000000,29.000000,509.000000,191.000000,80.000000,18.000000


In [46]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Sales        400 non-null    float64
 1   CompPrice    400 non-null    int64  
 2   Income       400 non-null    int64  
 3   Advertising  400 non-null    int64  
 4   Population   400 non-null    int64  
 5   Price        400 non-null    int64  
 6   ShelveLoc    400 non-null    object 
 7   Age          400 non-null    int64  
 8   Education    400 non-null    int64  
 9   Urban        400 non-null    object 
 10  US           400 non-null    object 
dtypes: float64(1), int64(7), object(3)
memory usage: 34.5+ KB


### **Find and Remove NA values**

In [47]:
data.isna().sum()

Sales          0
CompPrice      0
Income         0
Advertising    0
Population     0
Price          0
ShelveLoc      0
Age            0
Education      0
Urban          0
US             0
dtype: int64

# **Changing the categorical variables into dummies**

In [48]:
data = pd.get_dummies(data)

In [49]:
data.head()

,Sales,CompPrice,Income,Advertising,Population,Price,Age,Education,ShelveLoc_Bad,ShelveLoc_Good,ShelveLoc_Medium,Urban_No,Urban_Yes,US_No,US_Yes
0,9.50,138,73,11,276,120,42,17,1,0,0,0,1,0,1
1,11.22,111,48,16,260,83,65,10,0,1,0,0,1,0,1
2,10.06,113,35,10,269,80,59,12,0,0,1,0,1,0,1
3,7.40,117,100,4,466,97,55,14,0,0,1,0,1,0,1
4,4.15,141,64,3,340,128,38,13,1,0,0,0,1,1,0


In [50]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Sales             400 non-null    float64
 1   CompPrice         400 non-null    int64  
 2   Income            400 non-null    int64  
 3   Advertising       400 non-null    int64  
 4   Population        400 non-null    int64  
 5   Price             400 non-null    int64  
 6   Age               400 non-null    int64  
 7   Education         400 non-null    int64  
 8   ShelveLoc_Bad     400 non-null    uint8  
 9   ShelveLoc_Good    400 non-null    uint8  
 10  ShelveLoc_Medium  400 non-null    uint8  
 11  Urban_No          400 non-null    uint8  
 12  Urban_Yes         400 non-null    uint8  
 13  US_No             400 non-null    uint8  
 14  US_Yes            400 non-null    uint8  
dtypes: float64(1), int64(7), uint8(7)
memory usage: 27.9 KB


## **Set and Add new column as per problem statment**

In [51]:
company_val = []
for value in data["Sales"]:
     if value<=7.49:
        company_val.append("low")
     else:
        company_val.append("high")

data["Company_Val"]= company_val

In [52]:
data.head()

,Sales,CompPrice,Income,Advertising,Population,Price,Age,Education,ShelveLoc_Bad,ShelveLoc_Good,ShelveLoc_Medium,Urban_No,Urban_Yes,US_No,US_Yes,Company_Val
0,9.50,138,73,11,276,120,42,17,1,0,0,0,1,0,1,high
1,11.22,111,48,16,260,83,65,10,0,1,0,0,1,0,1,high
2,10.06,113,35,10,269,80,59,12,0,0,1,0,1,0,1,high
3,7.40,117,100,4,466,97,55,14,0,0,1,0,1,0,1,low
4,4.15,141,64,3,340,128,38,13,1,0,0,0,1,1,0,low


### **Drop 	Taxable_Income column**

In [53]:
data.drop(['Sales'], axis=1, inplace=True)

In [54]:
data.head(2)

,CompPrice,Income,Advertising,Population,Price,Age,Education,ShelveLoc_Bad,ShelveLoc_Good,ShelveLoc_Medium,Urban_No,Urban_Yes,US_No,US_Yes,Company_Val
0,138,73,11,276,120,42,17,1,0,0,0,1,0,1,high
1,111,48,16,260,83,65,10,0,1,0,0,1,0,1,high


### **Value check for y column**

In [55]:
data['Company_Val'].value_counts()

low     201
high    199
Name: Company_Val, dtype: int64

In [56]:
data.columns

Index(['CompPrice', 'Income', 'Advertising', 'Population', 'Price', 'Age',
       'Education', 'ShelveLoc_Bad', 'ShelveLoc_Good', 'ShelveLoc_Medium',
       'Urban_No', 'Urban_Yes', 'US_No', 'US_Yes', 'Company_Val'],
      dtype='object')

In [57]:
data.shape

(400, 15)

In [58]:
array = data.values
X = array[:,0:14]
Y = array[:,14]

In [59]:
num_trees = 100
max_features = 4 #max_features
seed = 7 #random_state

In [60]:
kfold = KFold(n_splits=10, random_state=7, shuffle = True)

# **Random Forest Classifier**

In [61]:
#  Random Forest Model Build
model = RandomForestClassifier(n_estimators=num_trees, max_features=max_features)

In [62]:
results = cross_val_score(model, X, Y, cv=kfold)

In [63]:
print("Random Forest Mode :",round(results.mean()*100,2))

Random Forest Mode : 80.5


# **Ensemble techniques**

## **1.Bagging**

In [64]:
cart = DecisionTreeClassifier()

In [65]:
model_Bagging = BaggingClassifier(base_estimator=cart, n_estimators=num_trees, random_state=seed)

In [66]:
results = cross_val_score(model_Bagging, X, Y, cv=kfold)


In [67]:
print("Bagging Model :",round(results.mean()*100,2))

Bagging Model : 80.75


## **2.Boosting**
AdaBoost Classification

In [68]:
model_Boosting = AdaBoostClassifier(n_estimators=num_trees, random_state=seed)

In [69]:
results = cross_val_score(model_Boosting, X, Y, cv=kfold)

In [70]:
print("Boosting Model :",round(results.mean()*100,2))

Boosting Model : 85.5


## **3.Stacking**
Stacking Ensemble for Classification

In [71]:
# create the sub models
estimators = [] # create empty list for different algorithms

In [72]:
model1 = LogisticRegression(max_iter=500)
estimators.append(('logistic', model1))

model2 = DecisionTreeClassifier()
estimators.append(('cart', model2))

model3 = SVC()
estimators.append(('svc', model3))

In [73]:
# create the ensemble model
ensemble = VotingClassifier(estimators)
results = cross_val_score(ensemble, X, Y, cv=kfold)

In [74]:
print('Stacking Model :',results.mean()*100)

Stacking Model : 84.25
